## 平行化 Function Calling

### 即時匯率計算

In [2]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
genai.configure(api_key=os.getenv("GENAI_API_KEY")) # 不知道從哪個版本起，都要把這行加上
model = genai.GenerativeModel("gemini-1.5-pro") # Function calling 建議採用最好的模型

In [3]:
from google.generativeai.types import FunctionDeclaration, Tool

get_exchange_rate_func = FunctionDeclaration(
    name = "get_exchange_rate",
    description = "Get the exchange rate for currencies between countries",
    parameters = {
        "type": "object",
        "properties": {
            "arg_amount": {
                "type": "number",
                "description": "The amount of money to convert"
            },
            "arg_from": {
                "type": "string",
                "description": "The currency to convert from in ISO 4217 format"
            },
            "arg_to": {
                "type": "string",
                "description": "The currency to convert to in ISO 4217 format"
            }
        },
        "required": [
            "arg_amount",
            "arg_from",
            "arg_to",
        ]
    },
)

exchange_rate_tool = Tool(
    function_declarations = [get_exchange_rate_func],
)
toolbox = [exchange_rate_tool]

In [5]:
prompt = "100美元可以兌換多少日圓和澳幣？"

messages = [{
    "role": "user",
    "parts": [{
        "text": prompt
    }]
}]

function_call_response = model.generate_content(
    messages,
    tools = toolbox,
)

function_call_response.candidates[0].content

parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "arg_to"
        value {
          string_value: "JPY"
        }
      }
      fields {
        key: "arg_from"
        value {
          string_value: "USD"
        }
      }
      fields {
        key: "arg_amount"
        value {
          number_value: 100
        }
      }
    }
  }
}
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "arg_to"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "arg_from"
        value {
          string_value: "USD"
        }
      }
      fields {
        key: "arg_amount"
        value {
          number_value: 100
        }
      }
    }
  }
}
role: "model"

In [6]:
import requests

def get_exchange_rate(args):
    params = {}
    for key, value in args.items():
        params[key[4:]] = value

    api_response = requests.get("https://api.frankfurter.app/latest", params = params)
    return api_response.text

for part in function_call_response.candidates[0].content.parts:
    print(get_exchange_rate(part.function_call.args))


{"amount":100.0,"base":"USD","date":"2024-07-19","rates":{"JPY":15758}}
{"amount":100.0,"base":"USD","date":"2024-07-19","rates":{"AUD":149.46}}


In [7]:
import json
from google.protobuf.struct_pb2 import Struct

content = function_call_response.candidates[0].content
messages.append(content)

function_parts = []
for part in content.parts:
    function_response = Struct()
    function_response.update(json.loads(get_exchange_rate(part.function_call.args)))
    function_parts.append({
        "function_response": {
            "name": part.function_call.name,
            "response": function_response
        }
    })

messages.append({
    "role": "function",
    "parts": function_parts
})

messages

[{'role': 'user', 'parts': [{'text': '100美元可以兌換多少日圓和澳幣？'}]},
 parts {
   function_call {
     name: "get_exchange_rate"
     args {
       fields {
         key: "arg_to"
         value {
           string_value: "JPY"
         }
       }
       fields {
         key: "arg_from"
         value {
           string_value: "USD"
         }
       }
       fields {
         key: "arg_amount"
         value {
           number_value: 100
         }
       }
     }
   }
 }
 parts {
   function_call {
     name: "get_exchange_rate"
     args {
       fields {
         key: "arg_to"
         value {
           string_value: "AUD"
         }
       }
       fields {
         key: "arg_from"
         value {
           string_value: "USD"
         }
       }
       fields {
         key: "arg_amount"
         value {
           number_value: 100
         }
       }
     }
   }
 }
 role: "model",
 {'role': 'function',
  'parts': [{'function_response': {'name': 'get_exchange_rate',
     'response'

In [8]:

response = model.generate_content(
    messages,
    tools = [exchange_rate_tool],
)

response.candidates[0].content.parts[0].text

'今天，2024 年 7 月 19 日，100 美元可以兌換 15758 日圓和 149.46 澳幣。'